In [1]:
import os
import time
import copy
import numpy as n
import numpy.lib.recfunctions as rfn
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix

import torch
from torch import nn

In [4]:
path = '/mnt/d/single-mouse-f1-Nov20-2020/results/Female1/VAME/kmeans-15'

In [3]:
path

'/mnt/d/single-mouse-f1-Nov20-2020'